In [ ]:
# COVID-19 Claim Verification System
# ===============================
# 
# This notebook implements a system for verifying COVID-19 related claims by:
# 1. Training a claim veracity prediction model on the enhanced COVID-Fact dataset
# 2. Implementing evidence retrieval using multiple approaches
# 3. Providing a simple interface for claim verification
#
# Author: [Samidur Rahman]
# Date: March 2025

import pandas as pd
import numpy as np
import json
import re
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# NLP libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    AutoModel,
    Trainer, 
    TrainingArguments
)
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sentence_transformers import SentenceTransformer
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import balanced_accuracy_score


# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Load spaCy model
try:
    nlp = spacy.load("en_core_web_sm")
except:
    # If not installed, download it
    os.system("python -m spacy download en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

# Set up device for PyTorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 1. Data Loading and Exploration
# ===============================

def load_jsonl_data(file_path):
    """Load data from a JSONL file"""
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# Load the enhanced dataset
print("Loading dataset...")
dataset_path = "../../claim-verification/data/processed/COVIDFACT_dataset.jsonl"
#dataset_path = "../../claim-verification/data/processed/filtered_dataset.jsonl"
data = load_jsonl_data(dataset_path)
print(f"Loaded {len(data)} claims")

# Convert to DataFrame for easier manipulation
df = pd.DataFrame(data)

# Display basic statistics
print("\nDataset Statistics:")
print(f"Total claims: {len(df)}")
print(f"Supported claims: {len(df[df['label'] == 'SUPPORTED'])}")
print(f"Refuted claims: {len(df[df['label'] == 'REFUTED'])}")

# Check for claims with generation_info (our enhanced counter-claims)
enhanced_claims = df[df.apply(lambda x: 'generation_info' in x, axis=1)]
print(f"Enhanced counter-claims: {len(enhanced_claims)}")



In [ ]:
# 2. Data Preprocessing
# =====================

def preprocess_claim(claim, remove_stopwords=False):
    """Preprocess a claim for feature extraction"""
    # Convert to lowercase
    claim = claim.lower()
    
    # Remove special characters and digits
    claim = re.sub(r'[^\w\s]', '', claim)
    claim = re.sub(r'\d+', '', claim)
    
    if remove_stopwords:
        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = word_tokenize(claim)
        filtered_tokens = [word for word in tokens if word not in stop_words]
        claim = ' '.join(filtered_tokens)
    
    return claim

# Function to prepare data for training
def prepare_data_for_training(df, text_cols=['claim'], label_col='label', test_size=0.2, preprocessed=False):
    """
    Prepare data for training by combining text columns and splitting into train/test sets
    
    Args:
        df: DataFrame containing the data
        text_cols: List of column names containing text to use as features
        label_col: Column name containing the labels
        test_size: Proportion of data to use for testing
        preprocessed: Whether to preprocess the text
        
    Returns:
        X_train, X_test, y_train, y_test: Training and testing data
    """
    # Combine text columns
    if len(text_cols) == 1:
        X = df[text_cols[0]].tolist()
    else:
        X = df.apply(lambda row: ' '.join([str(row[col]) for col in text_cols]), axis=1).tolist()
    
    # Preprocess text if needed
    if preprocessed:
        X = [preprocess_claim(text) for text in X]
    
    # Prepare labels
    y = df[label_col].map({'SUPPORTED': 1, 'REFUTED': 0}).tolist()
    
    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)
    
    return X_train, X_test, y_train, y_test

# Prepare the data for training
X_train, X_test, y_train, y_test = prepare_data_for_training(df)

print(f"\nTraining set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")

In [ ]:
# 3. Feature Engineering
# ======================

# 3.1 TF-IDF Features
print("\nExtracting TF-IDF features...")
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print(f"TF-IDF features shape: {X_train_tfidf.shape}")

# 3.2 Transformer-based Features
print("\nExtracting transformer-based features...")

def get_transformer_embeddings(texts, model_name='sentence-transformers/all-mpnet-base-v2'):
    """Extract embeddings from a transformer model"""
    model = SentenceTransformer(model_name)
    embeddings = model.encode(texts, show_progress_bar=True)
    return embeddings

# Extract transformer embeddings
X_train_transformer = get_transformer_embeddings(X_train)
X_test_transformer = get_transformer_embeddings(X_test)

print(f"Transformer embeddings shape: {X_train_transformer.shape}")

In [ ]:
# 4. Model Training
# ================

# 4.1 Traditional ML Models

# Function to train and evaluate a model
def train_evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """Train and evaluate a model"""
    print(f"\nTraining {model_name}...")
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['REFUTED', 'SUPPORTED'])
    
    print(f"{model_name} Accuracy: {accuracy:.4f}")
    print(f"{model_name} Classification Report:")
    print(report)
    
    # Create confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['REFUTED', 'SUPPORTED'],
                yticklabels=['REFUTED', 'SUPPORTED'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.show()
    
    return model

In [ ]:
# 4.1.1 Logistic Regression with TF-IDF
lr_model = LogisticRegression(max_iter=1000, C=1.0, class_weight='balanced')
lr_model = train_evaluate_model(lr_model, X_train_tfidf, X_test_tfidf, y_train, y_test, "Logistic Regression (TF-IDF)")

# 4.1.2 SVM with TF-IDF
svm_model = SVC(kernel='linear', C=1.0, probability=True, class_weight='balanced')
svm_model = train_evaluate_model(svm_model, X_train_tfidf, X_test_tfidf, y_train, y_test, "SVM (TF-IDF)")

# 4.1.3 Random Forest with TF-IDF
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model = train_evaluate_model(rf_model, X_train_tfidf, X_test_tfidf, y_train, y_test, "Random Forest (TF-IDF)")

# 4.1.4 Logistic Regression with Transformer embeddings
lr_transformer_model = LogisticRegression(max_iter=1000, C=1.0, class_weight='balanced')
lr_transformer_model = train_evaluate_model(lr_transformer_model, X_train_transformer, X_test_transformer, 
                                          y_train, y_test, "Logistic Regression (Transformer)")

In [ ]:
class ClaimVerifier:
    def __init__(self, model_name="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract", num_labels=2):
        """Initialize the claim verifier with a pre-trained model."""
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name, 
            num_labels=num_labels
        )
        
        # Better initialization for classification layer
        self.model.classifier.weight.data.normal_(mean=0.0, std=0.02)
        self.model.classifier.bias.data.zero_()
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        
    def prepare_dataset(self, claims, evidence_list, labels=None, max_length=384):
        """Prepare a dataset with claims and evidence."""
        # Create structured input: Claim followed by Evidence with clear separation
        texts = []
        for i, claim in enumerate(claims):
            evidence = evidence_list[i] if i < len(evidence_list) else ""
            
            # Format as structured input with special tokens for separation
            if isinstance(evidence, list):
                evidence_text = " ".join(evidence)
            else:
                evidence_text = str(evidence)
                
            # Use a template that clearly distinguishes claim from evidence
            combined_text = f"[CLAIM] {claim} [EVIDENCE] {evidence_text}"
            texts.append(combined_text)
        
        # Tokenize with special handling for longer sequences
        encodings = self.tokenizer(
            texts, 
            padding='max_length', 
            truncation=True, 
            max_length=max_length, 
            return_tensors=None
        )
        
        # Create a Dataset for the Trainer
        class VerificationDataset(Dataset):
            def __init__(self, encodings, labels=None):
                self.encodings = encodings
                self.labels = labels

            def __getitem__(self, idx):
                item = {key: val[idx] for key, val in self.encodings.items()}
                if self.labels is not None:
                    item["labels"] = self.labels[idx]
                return item

            def __len__(self):
                return len(self.encodings["input_ids"])
        
        return VerificationDataset(encodings, labels)
    
    def train(self, train_claims, train_evidence, train_labels, 
              eval_claims, eval_evidence, eval_labels,
              output_dir='claim_verification_model',
              epochs=5, batch_size=16, learning_rate=2e-5):
        """Train the model on claims with evidence."""
        
        # Prepare datasets
        train_dataset = self.prepare_dataset(train_claims, train_evidence, train_labels)
        eval_dataset = self.prepare_dataset(eval_claims, eval_evidence, eval_labels)
        
        # Calculate and print average text length for information
        avg_len = sum(len(self.tokenizer.encode(f"[CLAIM] {c} [EVIDENCE] {e if isinstance(e, str) else ' '.join(e)}")) 
                      for c, e in zip(train_claims[:100], train_evidence[:100])) / 100
        max_len = max(len(self.tokenizer.encode(f"[CLAIM] {c} [EVIDENCE] {e if isinstance(e, str) else ' '.join(e)}")) 
                     for c, e in zip(train_claims[:100], train_evidence[:100]))
        print(f"Average combined text length: {avg_len:.1f} tokens")
        print(f"Max combined text length: {max_len} tokens")
        
        # Calculate class weights to handle imbalance
        class_weights = compute_class_weight('balanced', 
                                            classes=np.unique(train_labels), 
                                            y=train_labels)
        class_weights_dict = {i: float(class_weights[i]) for i in range(len(class_weights))}
        print(f"Class weights: {class_weights_dict}")
        
        # Define metrics calculation with detailed logging
        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)
            
            # Log prediction distribution for debugging
            pred_distribution = {int(k): int(v) for k, v in 
                               zip(*np.unique(predictions, return_counts=True))}
            print(f"Prediction distribution: {pred_distribution}")
            
            precision, recall, f1, _ = precision_recall_fscore_support(
                labels, predictions, average='binary', zero_division=0
            )
            accuracy = accuracy_score(labels, predictions)
            
            # Calculate balanced accuracy for imbalanced datasets
            balanced_accuracy = balanced_accuracy_score(labels, predictions)
            
            return {
                'accuracy': accuracy,
                'balanced_accuracy': balanced_accuracy,
                'f1': f1,
                'precision': precision,
                'recall': recall
            }
        
        # Custom trainer with weighted loss
        class WeightedLossTrainer(Trainer):
            def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
                labels = inputs.pop("labels")
                outputs = model(**inputs)
                logits = outputs.logits
                
                # Use weighted loss for class imbalance
                weight = torch.tensor(
                    [class_weights_dict[i] for i in range(len(class_weights_dict))],
                    device=self.args.device
                )
                loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
                loss = loss_fct(logits.view(-1, self.model.config.num_labels), 
                               labels.view(-1))
                
                return (loss, outputs) if return_outputs else loss
        
        # Training arguments
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            gradient_accumulation_steps=2,  # Increase for stability
            learning_rate=learning_rate,
            weight_decay=0.01,
            warmup_ratio=0.1,
            fp16=torch.cuda.is_available(),
            evaluation_strategy="epoch",
            save_strategy="epoch",
            logging_dir=f"{output_dir}/logs",
            logging_steps=50,
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            greater_is_better=True,
            report_to="none",  # Disable wandb/tensorboard by default
        )
        
        # Initialize trainer
        trainer = WeightedLossTrainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            compute_metrics=compute_metrics,
        )
        
        # Train model
        trainer.train()
        
        # Final evaluation
        eval_results = trainer.evaluate()
        print(f"Final evaluation results: {eval_results}")
        
        # Save model
        self.model.save_pretrained(output_dir)
        self.tokenizer.save_pretrained(output_dir)
        
        return eval_results
    
    def predict(self, claims, evidence_list=None, batch_size=16):
        """Make predictions for claims, optionally with evidence."""
        # Prepare input in the same format as training
        texts = []
        for i, claim in enumerate(claims):
            if evidence_list and i < len(evidence_list):
                evidence = evidence_list[i]
                if isinstance(evidence, list):
                    evidence_text = " ".join(evidence)
                else:
                    evidence_text = str(evidence)
                combined_text = f"[CLAIM] {claim} [EVIDENCE] {evidence_text}"
            else:
                combined_text = f"[CLAIM] {claim} [EVIDENCE]"
            texts.append(combined_text)
        
        # Process in batches for efficiency
        results = {"labels": [], "probabilities": []}
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            
            # Tokenize
            inputs = self.tokenizer(
                batch_texts, 
                padding=True, 
                truncation=True, 
                return_tensors="pt"
            )
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            # Get predictions
            with torch.no_grad():
                outputs = self.model(**inputs)
            
            # Process results
            logits = outputs.logits
            probabilities = torch.nn.functional.softmax(logits, dim=-1)
            predictions = torch.argmax(logits, dim=-1)
            
            results["labels"].extend(predictions.cpu().numpy())
            results["probabilities"].extend(probabilities.cpu().numpy())
        
        return results
    
    

In [ ]:
# PubMedBERT is smaller than CT-BERT but specialized for biomedical text
verifier = ClaimVerifier(
    model_name="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract", 
    num_labels=2
)
# Load the dataset
dataset_path = "../../claim-verification/data/processed/filtered_dataset.jsonl"
data = load_jsonl_data(dataset_path)

# Extract claims, evidence, and labels
claims = [item['claim'] for item in data]
labels = [1 if item['label'] == 'SUPPORTED' else 0 for item in data]

# Create evidence mapping
claim_to_evidence = {}
for item in data:
    claim = item.get('claim', '')
    evidence = []
    if 'evidence' in item and isinstance(item['evidence'], list):
        evidence = item['evidence']
    elif 'evidence' in item and isinstance(item['evidence'], str):
        evidence = [item['evidence']]
    claim_to_evidence[claim] = evidence

# Split data into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    claims, labels, test_size=0.2, random_state=42, stratify=labels
)

# Map evidence to train and test claims
train_evidence = [claim_to_evidence.get(claim, []) for claim in X_train]
test_evidence = [claim_to_evidence.get(claim, []) for claim in X_test]


results = verifier.train(
    train_claims=X_train, 
    train_evidence=train_evidence,
    train_labels=y_train,
    eval_claims=X_test,
    eval_evidence=test_evidence, 
    eval_labels=y_test,
    output_dir='models/covid_claim_verification',
    batch_size=8,
    epochs=5,
    learning_rate=4e-6
)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
class MedicalClaimVerifier:
    """
    A model for verifying the veracity of medical claims using BioBERT.
    """
    
    def __init__(self, model_name="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract", num_labels=2):
        """
        Initialize the medical claim verifier.
        
        Args:
            model_name (str): The pretrained model to use
            num_labels (int): Number of classification labels (2 for binary)
        """
        self.model_name = model_name
        self.num_labels = num_labels
        
        # Load model and tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name, 
            num_labels=num_labels
        )
        
        # Better initialization for classification layer
        self.model.classifier.weight.data.normal_(mean=0.0, std=0.02)
        self.model.classifier.bias.data.zero_()
        
        # Set device
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        
        # For storing class weights
        self.class_weights = None
        
    def create_dataset(self, texts, labels=None, max_length=128):
        """
        Create a dataset from texts and optional labels.
        
        Args:
            texts (list): List of claim texts
            labels (list, optional): List of labels (1 for supported, 0 for refuted)
            max_length (int): Maximum sequence length for tokenization
            
        Returns:
            Dataset: A PyTorch dataset for training or inference
        """
        # Tokenize the texts
        encodings = self.tokenizer(
            texts, 
            padding='max_length', 
            truncation=True, 
            max_length=max_length, 
            return_tensors=None
        )
        
        # Create a dataset class
        class TextClassificationDataset(Dataset):
            def __init__(self, encodings, labels=None):
                self.encodings = encodings
                self.labels = labels

            def __getitem__(self, idx):
                item = {key: val[idx] for key, val in self.encodings.items()}
                if self.labels is not None:
                    item["labels"] = self.labels[idx]
                return item

            def __len__(self):
                return len(self.encodings["input_ids"])
        
        return TextClassificationDataset(encodings, labels)
    
    def compute_metrics(self, eval_pred):
        """
        Compute evaluation metrics.
        
        Args:
            eval_pred: Tuple of predictions and labels
            
        Returns:
            dict: Dictionary of metrics
        """
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        
        # Log prediction distribution
        pred_counts = np.bincount(predictions, minlength=self.num_labels)
        label_counts = np.bincount(labels, minlength=self.num_labels)
        
        print(f"Prediction distribution: {pred_counts}")
        print(f"Label distribution: {label_counts}")
        
        # Calculate metrics
        precision, recall, f1, _ = precision_recall_fscore_support(
            labels, predictions, average='binary', zero_division=0
        )
        accuracy = accuracy_score(labels, predictions)
        
        # Calculate class-specific metrics
        class_metrics = {}
        for cls in range(self.num_labels):
            cls_pred = (predictions == cls)
            cls_true = (labels == cls)
            cls_precision = np.sum(cls_pred & cls_true) / max(np.sum(cls_pred), 1)
            cls_recall = np.sum(cls_pred & cls_true) / max(np.sum(cls_true), 1)
            cls_f1 = 2 * cls_precision * cls_recall / max(cls_precision + cls_recall, 1e-8)
            class_metrics[f'class_{cls}_precision'] = float(cls_precision)
            class_metrics[f'class_{cls}_recall'] = float(cls_recall)
            class_metrics[f'class_{cls}_f1'] = float(cls_f1)
        
        metrics = {
            'accuracy': accuracy,
            'f1': f1,
            'precision': precision,
            'recall': recall,
            **class_metrics
        }
        
        return metrics
    
    def get_trainer(self, train_dataset, eval_dataset=None, training_args=None):
        """
        Create a trainer for the model.
        
        Args:
            train_dataset: Training dataset
            eval_dataset: Evaluation dataset
            training_args: TrainingArguments or None
            
        Returns:
            Trainer: A trainer for the model
        """
        # Create default training arguments if none provided
        if training_args is None:
            training_args = TrainingArguments(
                output_dir='./results',
                num_train_epochs=5,
                per_device_train_batch_size=16,
                per_device_eval_batch_size=16,
                warmup_ratio=0.1,
                weight_decay=0.01,
                logging_dir='./logs',
                logging_steps=50,
                evaluation_strategy='epoch',
                save_strategy='epoch',
                load_best_model_at_end=True,
                metric_for_best_model='f1',
                greater_is_better=True,
                learning_rate=5e-6,
                fp16=torch.cuda.is_available(),
            )
        
        # Create a custom trainer with weighted loss
        class WeightedLossTrainer(Trainer):
            def __init__(self, *args, class_weights=None, **kwargs):
                super().__init__(*args, **kwargs)
                self.class_weights = class_weights
                
            def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
                # Accepts any additional kwargs but doesn't use them
                labels = inputs.pop("labels")
                outputs = model(**inputs)
                logits = outputs.logits
                
                if self.class_weights is not None:
                    weight = torch.tensor(
                        self.class_weights,
                        device=model.device,
                        dtype=torch.float32
                    )
                    loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
                else:
                    loss_fct = torch.nn.CrossEntropyLoss()
                    
                loss = loss_fct(logits.view(-1, model.config.num_labels), labels.view(-1))
                
                return (loss, outputs) if return_outputs else loss
        
        # Create the trainer
        trainer = WeightedLossTrainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            compute_metrics=self.compute_metrics,
            class_weights=self.class_weights
        )
        
        return trainer
    
    def train(self, train_texts, train_labels, eval_texts=None, eval_labels=None, 
             output_dir='./model', batch_size=16, epochs=5, learning_rate=5e-6,
             max_length=128, gradient_accumulation_steps=2):
        """
        Train the model on medical claims.
        
        Args:
            train_texts (list): Training claim texts
            train_labels (list): Training labels
            eval_texts (list, optional): Evaluation claim texts
            eval_labels (list, optional): Evaluation labels
            output_dir (str): Directory to save the model
            batch_size (int): Batch size for training
            epochs (int): Number of training epochs
            learning_rate (float): Learning rate
            max_length (int): Maximum sequence length
            gradient_accumulation_steps (int): Number of steps for gradient accumulation
            
        Returns:
            dict: Evaluation results
        """
        # Calculate class weights for imbalanced data
        unique_labels = np.unique(train_labels)
        self.class_weights = compute_class_weight(
            'balanced', 
            classes=unique_labels, 
            y=train_labels
        ).tolist()
        
        print(f"Class weights: {dict(zip(unique_labels, self.class_weights))}")
        
        # Create datasets
        train_dataset = self.create_dataset(train_texts, train_labels, max_length)
        eval_dataset = None
        if eval_texts is not None and eval_labels is not None:
            eval_dataset = self.create_dataset(eval_texts, eval_labels, max_length)
        
        # Create training arguments
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            warmup_ratio=0.1,
            weight_decay=0.01,
            logging_dir=f'{output_dir}/logs',
            logging_steps=50,
            evaluation_strategy='epoch',
            save_strategy='epoch',
            load_best_model_at_end=True,
            metric_for_best_model='f1',
            greater_is_better=True,
            learning_rate=learning_rate,
            fp16=torch.cuda.is_available(),
        )
        
        # Get trainer
        trainer = self.get_trainer(train_dataset, eval_dataset, training_args)
        
        # Train the model
        trainer.train()
        
        # Evaluate
        eval_results = {}
        if eval_dataset:
            eval_results = trainer.evaluate()
            print(f"Evaluation results: {eval_results}")
        
        # Save the model
        self.model.save_pretrained(output_dir)
        self.tokenizer.save_pretrained(output_dir)
        
        return eval_results
    
    def predict(self, texts, batch_size=16, max_length=128):
        """
        Make predictions on new claims.
        
        Args:
            texts (list): List of claim texts
            batch_size (int): Batch size for prediction
            max_length (int): Maximum sequence length
            
        Returns:
            dict: Dictionary with predictions and probabilities
        """
        # Create a dataset without labels
        dataset = self.create_dataset(texts, max_length=max_length)
        
        # Create a simple trainer for prediction
        trainer = Trainer(
            model=self.model,
            args=TrainingArguments(
                output_dir='./temp_predict',
                per_device_eval_batch_size=batch_size,
                report_to="none"
            )
        )
        
        # Get raw predictions
        raw_predictions = trainer.predict(dataset)
        logits = raw_predictions.predictions
        
        # Convert to probabilities and labels
        probabilities = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
        predictions = np.argmax(logits, axis=-1)
        
        # Map to readable format
        label_map = {0: "REFUTED", 1: "SUPPORTED"}
        readable_predictions = [label_map[pred] for pred in predictions]
        
        # Create confidence scores (probability of predicted class)
        confidence = [float(prob[pred]) for prob, pred in zip(probabilities, predictions)]
        
        return {
            "predictions": predictions.tolist(),
            "readable_predictions": readable_predictions,
            "probabilities": probabilities.tolist(),
            "confidence": confidence
        }
    
    @classmethod
    def load(cls, model_path):
        """
        Load a trained model from disk.
        
        Args:
            model_path (str): Path to the saved model
            
        Returns:
            MedicalClaimVerifier: A loaded model instance
        """
        instance = cls()
        instance.tokenizer = AutoTokenizer.from_pretrained(model_path)
        instance.model = AutoModelForSequenceClassification.from_pretrained(model_path)
        instance.model.to(instance.device)
        return instance

In [ ]:
# Initialize the verifier
verifier = MedicalClaimVerifier()

# Train the model
results = verifier.train(
    train_texts=X_train, 
    train_labels=y_train,
    eval_texts=X_test, 
    eval_labels=y_test,
    output_dir='medical_claim_verifier',
    batch_size=8,
    epochs=5,
    learning_rate=4e-6
)